### Upload batch file

In [4]:
from dotenv import load_dotenv
import os
from openai import AzureOpenAI

load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT")
    )

# Upload a file with a purpose of "batch"
file = client.files.create(
  file=open("batch.jsonl", "rb"), 
  purpose="batch"
)

print(file.model_dump_json(indent=2))
file_id = file.id

{
  "id": "file-4c9fad0c02e841e49f2471f1a848b6db",
  "bytes": 820,
  "created_at": 1722950631,
  "filename": "batch.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "pending",
  "status_details": null
}


### Track file upload status

In [5]:
# Wait until the uploaded file is in processed state
import time
import datetime 

status = "pending"
while status != "processed":
    time.sleep(15)
    file_response = client.files.retrieve(file_id)
    status = file_response.status
    print(f"{datetime.datetime.now()} File Id: {file_id}, Status: {status}")

2024-08-06 14:24:18.219036 File Id: file-4c9fad0c02e841e49f2471f1a848b6db, Status: processed


### Create batch job

In [6]:
# Submit a batch job with the file
batch_response = client.batches.create(
    input_file_id=file_id,
    endpoint="/chat/completions",
    completion_window="24h",
)

# Save batch ID for later use
batch_id = batch_response.id

print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_aa63aaa8-0308-47c8-8fe6-c155617023f9",
  "completion_window": "24h",
  "created_at": 1722950663,
  "endpoint": "/chat/completions",
  "input_file_id": "file-4c9fad0c02e841e49f2471f1a848b6db",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": null,
  "expired_at": null,
  "expires_at": 1723037063,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


### Track batch job progress

In [7]:
import time
import datetime 

status = "validating"
while status not in ("completed", "failed", "canceled"):
    time.sleep(60)
    batch_response = client.batches.retrieve(batch_id)
    status = batch_response.status
    print(f"{datetime.datetime.now()} Batch Id: {batch_id},  Status: {status}")

2024-08-06 14:26:05.435185 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Status: validating
2024-08-06 14:27:06.291753 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Status: validating
2024-08-06 14:28:07.128467 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Status: in_progress
2024-08-06 14:29:07.711255 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Status: in_progress
2024-08-06 14:30:08.696466 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Status: in_progress
2024-08-06 14:31:09.389244 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Status: in_progress
2024-08-06 14:32:10.082145 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Status: in_progress
2024-08-06 14:33:10.825973 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Status: in_progress
2024-08-06 14:34:11.774643 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Status: finalizing
2024-08-06 14:35:12.393370 Batch Id: batch_aa63aaa8-0308-47c8-8fe6-c155617023f9,  Sta

#### Examine job status details

In [9]:
print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_aa63aaa8-0308-47c8-8fe6-c155617023f9",
  "completion_window": "24h",
  "created_at": 1722950663,
  "endpoint": "/chat/completions",
  "input_file_id": "file-4c9fad0c02e841e49f2471f1a848b6db",
  "object": "batch",
  "status": "completed",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": 1722951376,
  "error_file_id": "file-8ba1cd8e-fca5-41f0-9cc9-264d3ce28fac",
  "errors": null,
  "expired_at": null,
  "expires_at": 1723037063,
  "failed_at": null,
  "finalizing_at": 1722951218,
  "in_progress_at": 1722950862,
  "metadata": null,
  "output_file_id": "file-05d6a16c-612a-426b-9370-785f37ed7716",
  "request_counts": {
    "completed": 3,
    "failed": 0,
    "total": 3
  }
}


### Retrieve batch job output file

In [11]:
import json

response = client.files.content(batch_response.output_file_id)
raw_responses = response.text.strip().split('\n')  

for raw_response in raw_responses:  
    json_response = json.loads(raw_response)  
    formatted_json = json.dumps(json_response, indent=2)  
    print(formatted_json)

{
  "custom_id": "task-1",
  "response": {
    "body": {
      "choices": [
        {
          "content_filter_results": {
            "hate": {
              "filtered": false,
              "severity": "safe"
            },
            "protected_material_code": {
              "filtered": false,
              "detected": false
            },
            "protected_material_text": {
              "filtered": false,
              "detected": false
            },
            "self_harm": {
              "filtered": false,
              "severity": "safe"
            },
            "sexual": {
              "filtered": false,
              "severity": "safe"
            },
            "violence": {
              "filtered": false,
              "severity": "safe"
            }
          },
          "finish_reason": "stop",
          "index": 0,
          "logprobs": null,
          "message": {
            "content": "The first Xbox was released on November 15, 2001.",
            "ro

### Additional batch commands

#### Cancel batch

In [ ]:
client.batches.cancel("batch_abc123") # set to your batch_id for the job you want to cancel

#### List batch

In [ ]:
client.batches.list()